In [1]:
from numba import jit

In [14]:
import numpy as np
import pickle

In [19]:
from power_planner.utils import get_half_donut, angle

In [16]:
instance_path = "/Users/ninawiedemann/Downloads/tifs_new/data_dump_5.dat"
with open(instance_path, "rb") as infile:
    data = pickle.load(infile)
(instance, instance_corr, start_inds, dest_inds) = data.data

min_dist = 3
max_dist = 5
weights = [0.5,0.5]
test_example = np.mean(instance, axis=0)

In [11]:
@jit
def shift(arr):
    return np.roll(arr, 2)

In [12]:
shift(np.arange(20).reshape(4,5))

array([[18, 19,  0,  1,  2],
       [ 3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12],
       [13, 14, 15, 16, 17]])

In [49]:
@jit
def compute_dists(n_iters, shifts, test_example, start_inds, dest_inds, angles_all, angle_weight=0.5):
    # weights = [0.5,0.5]
    min_angle= 1 / 4 # TODO

    fill_val = np.inf
    # one iteration
    # for i in range(len(shifts)):
    x_len, y_len = test_example.shape
    
    dists = np.zeros((len(shifts), x_len, y_len))
    dists += fill_val
    dists[:, start_inds[0], start_inds[1]] = test_example[start_inds[0], start_inds[1]]
    dists_argmin = np.zeros(dists.shape)
    # precompute angles
    
    # print(angles_all)
    for j in range(n_iters):
        for i in range(len(shifts)): # 10,11): # 
            # i = 10
            # compute angle from the sorted neighbor list to the new edge
            # immer plus das minimum
            curr_shift = shifts[i]

            angles = angles_all[i]
            # shift dists by this shift
            # todo: avoid swaping dimenions each time
            # cost_switched = np.swapaxes(dists, 1, 0)
            # shift by shift
            costs_shifted = dists.copy() # cost_switched
            # shift_surface(
            #     cost_switched, curr_shift
            # )
            # costs_shifted[costs_shifted==0] = fill_val

            # POSSIBLE BUT not necessary!
            # test_shifted = shift_surface(
            #     test_example.copy(), curr_shift, fill_val=0
            # )
            # norm_angle = angles/min_angle # np.max(angles)
            # norm_angle[norm_angle>min_angle] = np.inf
            # norm_angle>1 --> np.inf
            
            # costs_shifted = np.moveaxis(costs_shifted, -1, 0)
            together = np.zeros(costs_shifted.shape)
            # for k in range(len(shifts)):
            #     for l in range(x_len):
            #         for m in range(y_len):
            #             together[k,l,m] = costs_shifted[k,l,m] + angles[k] * angle_weight + test_example[l,m]* resistance_weight
          
            angle_cost = angle_weight *  angles
            together = costs_shifted + test_example
            # 28 x 10 x 10 + 28 angles + 10 x 10
            weighted_costs_shifted = np.min(together, axis=0)
            
            # if j==8 and i==16:
            #     print(angle_cost)
            #     print(np.argmin(together, axis=0)[16,70])
            #     print(weighted_costs_shifted[16, 70])
            # 

            concat = np.array([dists[i], weighted_costs_shifted])
            
            # temp = dists_argmin[i]
            # print(temp.shape)
            # get spots that are actually updated
            changed_ones = np.argmin(concat, axis=0)
            # get argmin for each edge --> remember where the value on this edge came from
            argmin_together = np.argmin(together, axis=0)
            ## FOR LOOP SOLUTION
            # for m in range(len(temp)):
            #     for n in range(len(temp[0])):
            #         if changed_ones[m,n]:
            #             dists_argmin[i,m,n] = argmin_together[m,n]
            ## TEMP SOLUTION:
            # temp[changed_ones>0] = argmin_together[changed_ones>0]
            # dists_argmin[i] = temp
            dists_argmin[i, changed_ones>0] = argmin_together[changed_ones>0]
            
            dists[i] = np.min(concat, axis=0)
            # print("-------------")
    assert(np.all(dists_argmin<np.inf))
    print("opt", np.min(dists[:, dest_inds[0], dest_inds[1]]))
    
    return dists, dists_argmin



In [50]:
# def my_path_computation(n_iters, shifts, test_example, start_inds, dest_inds, weights = [0.5, 1]):
shifts_ab = get_half_donut(3,5, dest_inds-start_inds, angle_max = 0.5*np.pi)
angles_all = np.zeros((len(shifts_ab), len(shifts_ab)))
for i in range(len(shifts_ab)):
    angles_all[i] = [angle(s, shifts_ab[i]) for s in shifts_ab]
angles_all = angles_all/(np.pi*0.5)
angles_all[angles_all>1] = np.inf
dists_ab, dists_argmin_ab = compute_dists(50, shifts_ab, test_example, start_inds, dest_inds, angles_all)

# todo for numba:
# angles weg
# time.time
# moveaxis and swapaxes

<ipython-input-49-f5409f320114>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "compute_dists" failed type inference due to: Invalid use of Function(<function amin at 0x1148a3320>) with argument(s) of type(s): (array(float64, 3d, C), axis=Literal[int](0))
 * parameterized
In definition 0:
    TypingError: numba doesn't support kwarg for min
    raised from /Users/ninawiedemann/Desktop/thesis.nosync/power_planner/powerenv/lib/python3.7/site-packages/numba/typing/npydecl.py:402
In definition 1:
    TypingError: numba doesn't support kwarg for min
    raised from /Users/ninawiedemann/Desktop/thesis.nosync/power_planner/powerenv/lib/python3.7/site-packages/numba/typing/npydecl.py:402
This error is usually caused by passing an argument of a type that is unsupported by the named function.
[1] During: resolving callee type: Function(<function amin at 0x1148a3320>)
[2] During: typing of call at <ipython-input-49-f5409f320114> (54)


File 

opt inf


In [45]:
@jit
def nump(arr):
    return np.swapaxes(arr, 1, 0)

In [46]:
nump(np.arange(20).reshape(4,5))

<ipython-input-45-ccc60a4a4ae1>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "nump" failed type inference due to: Use of unsupported NumPy function 'numpy.swapaxes' or unsupported use of the function.

File "<ipython-input-45-ccc60a4a4ae1>", line 3:
def nump(arr):
    return np.swapaxes(arr, 1, 0)
    ^

[1] During: typing of get attribute at <ipython-input-45-ccc60a4a4ae1> (3)

File "<ipython-input-45-ccc60a4a4ae1>", line 3:
def nump(arr):
    return np.swapaxes(arr, 1, 0)
    ^

  @jit
/Users/ninawiedemann/Desktop/thesis.nosync/power_planner/powerenv/lib/python3.7/site-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "nump" was compiled in object mode without forceobj=True.

File "<ipython-input-45-ccc60a4a4ae1>", line 2:
@jit
def nump(arr):
^

  state.func_ir.loc))
/Users/ninawiedemann/Desktop/thesis.nosync/power_planner/powerenv/lib/python3.7/site-packages/numba/object_mode_passes.py:188: NumbaDeprecationWarn

array([[ 0,  5, 10, 15],
       [ 1,  6, 11, 16],
       [ 2,  7, 12, 17],
       [ 3,  8, 13, 18],
       [ 4,  9, 14, 19]])